In [21]:
import tensorflow as tf 
from classifiers import PhasedClassifier, LSTMClassifier
import data
from sklearn.metrics import balanced_accuracy_score, accuracy_score, f1_score
import numpy as np

In [22]:
dataset = 'wise'
fold_n  = 1
rnn_unit = 'phased'
units = 256

In [23]:
path_exp = './experiments/{}/fold_{}/{}_{}/'.format(dataset, fold_n, rnn_unit, units)
path_data = '../datasets/records/{}/fold_{}/test.tfrecords'.format(dataset, fold_n)
path_ckpts = '{}/{}'.format(path_exp, 'ckpts/')

In [24]:
test_batches = data.load_record(path=path_data, batch_size=5000)
n_classes = [len(b[1][0]) for b in test_batches.take(1)][0]

In [25]:
if rnn_unit == 'phased':
    model = PhasedClassifier(units=units, 
                             n_classes=n_classes,
                             use_old=False)
else:
    model = LSTMClassifier(units=units, 
                           n_classes=n_classes)

In [26]:
model

In [27]:
model.load_ckpt(path_ckpts)

RNN Restored!


In [28]:
def fix_pred(y_pred, test_batches):
    masks = []
    for t in test_batches:
        masks.append(t[2])
    masks = tf.concat(masks, 0)
    valid_mask = tf.cast(tf.reduce_sum(masks, 1), dtype=tf.bool)
    y_pred = tf.boolean_mask(y_pred, valid_mask)
    return y_pred

In [29]:
y_pred, y_true, loss_test = model.predict_proba(test_batches)

runtime 179.89


In [30]:
y_last_pred = y_pred[:,-1,:]

In [31]:
print(y_true.shape)
y_true_fixed = fix_pred(y_true, test_batches)
print(y_true_fixed.shape)

(30538, 9)
(30537, 9)


In [40]:
labels_pred = np.argmax(y_last_pred.numpy()[:20], 1)
labels_true = np.argmax(y_true_fixed.numpy()[:20], 1)

In [41]:
accuracy_score(labels_true, labels_pred)

0.15

In [34]:
f1_score(labels_true, labels_pred, average='macro')

0.03029615115974417